In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from kiwipiepy import Kiwi
from hanspell import spell_checker

tqdm.pandas()

In [2]:
df = pd.read_csv('comment_all.csv',encoding='utf-8-sig')

In [3]:
# df = df[:10000]

In [4]:
kiwi = Kiwi(model_type='sbg',num_workers=0, typos='basic')

In [5]:
from kiwipiepy.utils import Stopwords
stopwords = Stopwords()

In [6]:
TB = pd.read_csv('result_day_TB_7.csv', encoding='utf-8-sig')

In [7]:
def kiwi_add(data, kiwi):
    data['NNP'] = np.nan
    data['NNP'] = data['NNP'].fillna('NNP')
    data = data[['word','NNP','finscore']]
    result = []
    
    for i in range(len(data)):
        result.append(kiwi.add_user_word(data['word'][i],data['NNP'][i],data['finscore'][i]))
    return result

In [8]:
TB = TB.rename(columns = {'0':'word','1':'finscore'})

In [9]:
kiwi_add(TB,kiwi)

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [10]:
# kiwi.prepare()

In [11]:
def cleansing(body, mode=None):
        body = re.sub(r'[^ A-Za-z가-힣+]',' ' , body, 0, re.I|re.S) # 한글, 알파벳을 제외한 나머지 제거       
        body = re.sub(r'(?:\b[0-9a-zA-Zㄱ-ㅎㅏ-ㅣ]\b|[?!\W]+)\s*', ' ', body).strip() 
        body = re.sub('<.+?>', ' ', body, 0, re.I|re.S)
        
        n=1
        for i in body:
            body = re.sub(f'(\\S{{{n},}}?)\\1+', ' ', body)

        if mode == 'quot':
            return body
        else:
            return ''.join(body)

In [12]:
df = df.comment.dropna()

In [13]:
df = pd.DataFrame(df)

In [14]:
df_comment = df.comment.progress_apply(cleansing)

100%|██████████| 273626/273626 [01:47<00:00, 2551.29it/s]


In [15]:
def spell(data):
    spelled_sent = spell_checker.check(data)
    hanspell_sent = spelled_sent.checked
    return hanspell_sent

In [16]:
df_comment = pd.DataFrame(df_comment)

In [17]:
df_comment1 = df_comment.drop(df_comment[df_comment['comment'] == ''].reset_index()['index'])

In [18]:
df_comment1

,comment
0,소방관 했는데 쓰레기도 있네
1,역시 전라도
2,이짝동네 사람들은 진짜 과학이다 과학여 완장을차나 감투만 쓰면 갑질을 해대니
3,당장이라도 짤라버리고 사법처리 해야 합니다 완전히 진상이네
4,대깨문 특징 갑질
...,...
273622,뭔 기사가 이러냐 현장에 기업의불법이 만연하고 있는 거잖아 그걸 바꾸지 않으면 강경...
273623,월 이라는단어만듣고노조니어쩌니하는거는웃긴거같음 돈을많이받는거는그만큼힘들고목숨걸고하는...
273624,타워크레인 기사가 이 많은건가 지들 술한번 사 쳐먹을 돈인데
273625,윤석열의 노조 악마화하기 기획 기사군요


In [19]:
df_comment1 = df_comment1.drop_duplicates().reset_index().drop('index',axis=1)

In [20]:
df_comment2 = []
for i in tqdm(df_comment1.comment):
        try:
            df_comment2.append(spell(i))
        except:
            pass

100%|██████████| 196879/196879 [3:47:59<00:00, 14.39it/s]  


In [21]:
len(df_comment2)

196876

In [22]:
df_comment2 = pd.DataFrame(df_comment2)

In [23]:
df_comment2.tail(50)

,0
196826,한국 최대 조폭집단 민노총 민주를 외치나 실제 목적은 돈 최대 양아치 집단
196827,민노총 한노총은 노동자의 대표가 아니다 사회악의 축면 합법을 가장한 깡패 집단이니 ...
196828,떼거지로 몰려다니면서 지뱃속채울려는 노조 새끼들은 강력하게 처벌해 본때를 보여줘야 ...
196829,이것들도 도박비 부족한 갑니다 일은 안 하고 맨날 파업이나 하는 거머리 같은 놈들이
196830,그런 건 협박이죠 당근 수사해서 감옥 처넣어야지 근데 노조가 기업보다 세다고 그건 ...
196831,개 jo 같은 Se 끼들 이러니 나라가 이 모양이지 저 잡것들 자격증 영구 취소 재...
196832,문죄 앙리가 세월호 키워 성역 만들어 양아치 노조 귀족 만들어 북한에 돈 주며 달래...
196833,삼성을 국유화하면 국민 모두가 잘 먹고 잘 살 수 있다고 하는 더불어 당 민노총 좌...
196834,와 조폭보다 더 양아치들이네 민노총만 일망타진해도 윤 지부는 절반의 성공이다
196835,월천 대사


In [24]:
f = open(f'new_dict.txt','w',encoding='utf-8-sig')
for i in df_comment2[0]:
            f.write(str(i))
f.close()
inputs = list(open(f'new_dict.txt', encoding='utf-8-sig'))       
reuslt = kiwi.extract_words(inputs, min_cnt=10, max_word_len=7, min_score=0.25)

In [25]:
pd.DataFrame(reuslt)[0].tolist()

['방음터널',
 '비례대표',
 '국정조사',
 '안전불감증',
 '전기차',
 '내로남불',
 '컨트롤타워',
 '드라이비트',
 '다대한민국',
 '소시오패스',
 '다방음터널',
 '다화물연대',
 '지하주차장',
 '용산구청장',
 '검찰공화국',
 '증거인멸',
 '공영방송',
 '아크로비스타',
 '알코올중독자',
 '다국정조사',
 '개인사업자',
 '소방서장',
 '다구급대원',
 '다우리나라',
 '자해공갈단',
 '유가족협의회',
 '둔촌주공',
 '한걸레',
 '살인미수',
 '과잉진료',
 '다비례대표',
 '직권남용',
 '치과의사',
 '다국회의원',
 '과유불급',
 '공무집행방해',
 '귀족노조',
 '라용산구청장',
 '다민노총',
 '내연기관차',
 '다윤석열']

In [26]:
kiwi.typo_cost_weight = 2

In [27]:
morph_analysis = lambda x: kiwi.tokenize(x,stopwords=stopwords) if type(x) is str else None

In [28]:
result = df_comment2[0].progress_apply(morph_analysis)

100%|██████████| 196876/196876 [09:07<00:00, 359.64it/s]


In [29]:
result = pd.DataFrame(result)

In [30]:
result

,0
0,"[(소방관, NNG, 0, 3), (데, NNB, 8, 1), (쓰레기, NNG, ..."
1,"[(역시, MAG, 0, 2), (전라도, NNP, 3, 3)]"
2,"[(짝, NNG, 1, 1), (동네, NNG, 3, 2), (진짜, MAG, 11..."
3,"[(당장, NNG, 0, 2), (라도, EC, 3, 2), (자르, VV, 6, ..."
4,"[(대, XPN, 0, 1), (깨물, VV, 1, 2), (특징, NNG, 4, ..."
...,...
196871,"[(뭔, MM, 0, 1), (기사, NNG, 2, 2), (이러, VV, 6, 2..."
196872,"[(월, NNG, 0, 1), (단어, NNG, 5, 2), (듣, VV-I, 9,..."
196873,"[(타워, NNG, 0, 2), (크레인, NNG, 2, 3), (기사, NNG, ..."
196874,"[(윤석열, NNP, 0, 3), (노조, NNG, 5, 2), (악마, NNG, ..."


In [31]:
from collections import Counter
'''주요 품사, 용언 품사 정의'''
주요품사 = ['NNG', 'NNP', 'VV', 'VA', 'XR', 'SL']
용언품사 = ['VV', 'VA']
'''Counter를 활용해 가장 많이 나온 n개의 품사 결과를 돌려주는 pos_count() 함수'''
def pos_count(df, col, output_filename, n=1000000):
    카운터 = Counter()
    
    for index, row in df.iterrows(): 
        if row[col]:
            필터링결과 = [(token.form, token.tag) for token in row[col] if token.tag in 주요품사]
            카운터.update(필터링결과)
            
    with open(output_filename, "w", encoding='utf-8-sig') as output_file:
        print("형태소,품사,개수", file=output_file)
        for (형태소, 품사), 개수 in 카운터.most_common(n):
            if 품사 in 용언품사:
                형태소 += "다"
            print(f"{형태소},{품사},{개수}", file=output_file)

In [32]:
pos_count(result,0,'commnet_token.csv')

In [33]:
comment_token = pd.read_csv('commnet_token.csv',encoding='utf-8-sig')

In [34]:
comment_token.head(50)

,형태소,품사,개수
0,가다,VV,18501
1,사고,NNG,15242
2,국민,NNG,14746
3,있다,VA,12959
4,나라,NNG,12305
5,나다,VV,11890
6,민주당,NNP,11359
7,정부,NNG,9856
8,죽다,VV,9483
9,차,NNG,9187


In [35]:
cond1 = comment_token.품사 == 'VV'
cond2 = comment_token.품사 == 'VA'
comment_token[cond1|cond2].head(50)

,형태소,품사,개수
0,가다,VV,18501
3,있다,VA,12959
5,나다,VV,11890
8,죽다,VV,9483
10,알다,VV,8745
11,살다,VV,7780
14,만들다,VV,7324
17,쓰다,VV,6971
19,많다,VA,6750
21,좋다,VA,6426


In [36]:
cond1 = comment_token['품사'] == 'NNG'
cond2 = comment_token['품사'] == 'NNP'

In [37]:
comment_token.loc[cond1 | cond2]['형태소'].tolist()

['사고',
 '국민',
 '나라',
 '민주당',
 '정부',
 '차',
 '안전',
 '책임',
 '대통령',
 '생각',
 '돈',
 '경찰',
 '이태원',
 '참사',
 '인간',
 '정권',
 '문제',
 '조사',
 '국가',
 '윤',
 '법',
 '탓',
 '정치',
 '기사',
 '불',
 '윤석열',
 '장관',
 '터널',
 '의원',
 '전기',
 '대한민국',
 '화재',
 '시간',
 '민노총',
 '개',
 '이상',
 '현장',
 '세금',
 '집',
 '당',
 '문재인',
 '노조',
 '소리',
 '앞',
 '정신',
 '파업',
 '수사',
 '재앙',
 '소방',
 '사건',
 '전',
 '운전',
 '쓰레기',
 '필요',
 '공무원',
 '남',
 '구속',
 '애',
 '용산',
 '이번',
 '술',
 '좌파',
 '국',
 '죄',
 '짓',
 '소방관',
 '힘',
 '처벌',
 '곳',
 '국회',
 '사진',
 '정도',
 '기업',
 '유가족',
 '국정',
 '운임',
 '구조',
 '피해',
 '세월호',
 '이재명',
 '사퇴',
 '세상',
 '이상민',
 '상황',
 '의사',
 '배',
 '경제',
 '검찰',
 '이용',
 '사과',
 '문',
 '시민',
 '차량',
 '선동',
 '도로',
 '유족',
 '중국',
 '잘못',
 '자식',
 '언론',
 '물',
 '본인',
 '댓글',
 '아파트',
 '방음',
 '사회',
 '한국',
 '날',
 '눈',
 '후',
 '자리',
 '이유',
 '발생',
 '위험',
 '불법',
 '입',
 '무능',
 '수준',
 '지금',
 '명복',
 '답',
 '뒤',
 '꼴',
 '압사',
 '대표',
 '북한',
 '위',
 '기자',
 '진상',
 '집단',
 '부모',
 '관리',
 '고인',
 '고생',
 '음주',
 '거지',
 '동안',
 '탄핵',
 '끝',
 '이제',
 '행안부',
 '보상',
 '가족',
 '단체',
 '아이',